# Testing masked arrays with donuts

To add masking to Donuts arrays we want to explore making all arrays masked arrays and then supplying a mask or not, depending on the given imager. 

To implement this change we need to do the following:

   1. Pull out the main array operations of Donuts
   1. Add an option to supplying a boolean (True/False or 1/0) array of the same shape as the raw image (including any pre/overscan)
   1. Immediately after loading an image, apply the mask (or not)
   1. Test run all donuts operations for support of masked arrays
   
A very simple test could be to manually load an image, mask it, then run the methods of the ```donuts.image.Image``` class without any changes to see what happens. 

### Update:

It seems like all the functionality of numpy just works with masked arrays, see below. 

The changes required to Donuts are:
   1. Add an option to take in a mask (must be same shape as raw image array)
   1. Make all arrays masked arrays after loading the data.
   1. Apply the mask or supply a null mask if no masking is needed
   1. Proceed as normal

In [1]:
import os
import glob as g
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from donuts import Donuts
from donuts.image import Image

In [2]:
os.chdir('/Users/jmcc/Dropbox/data/ngts/action293930_observeField')
imgs = sorted(g.glob("*.fits"))

In [ ]:
# work on reference image
ref_img = imgs[0]

with fits.open(ref_img) as ff:
    data = ff[0].data
    header = ff[0].header

# make a copy of the original data
data_orig = np.copy(data)

# make a copy to play with for masking
data_copy = np.copy(data)

# add a massive spikes to some pixels
data_copy[500:510, 500:510] += 60000
data_copy[1500:1510, 1500:1510] += 60000
data_copy[1000:1010, 1000:1010] += 60000


# make a mask to exclude 500:510, 500:510
data_copy_mask = np.zeros((data.shape))
data_copy_mask[500:510, 500:510] = 1
data_copy_mask[1500:1510, 1500:1510] = 1
data_copy_mask[1000:1010, 1000:1010] = 1
data_copy_ma = np.ma.array(data_copy, mask=data_copy_mask, fill_value=0)

print(data_orig[500:510, 500:510])
print(data_copy[500:510, 500:510])
print(data_copy_ma[500:510, 500:510])

# display the three images 
plt.imshow(data_orig)
plt.show()
plt.imshow(data_copy)
plt.show()
plt.imshow(data_copy_ma)
plt.show()

In [ ]:
# instanciate the donuts.image.Image class with the original unmasked data
ref_image_orig = Image(data_orig, header)
clyo, cuyo, clxo, cuxo = ref_image_orig.calculate_image_geometry(20, 20, 'x', 65, 32)
ref_image_orig.trim(clyo, cuyo, clxo, cuxo)
ref_image_orig.remove_background(32)
ref_image_orig.downweight_edges()
ref_image_orig.compute_projections()

# instanciate the donuts.image.Image class with the modified data
ref_image_copy = Image(data_copy, header)
clyc, cuyc, clxc, cuxc = ref_image_copy.calculate_image_geometry(20, 20, 'x', 65, 32)
ref_image_copy.trim(clyc, cuyc, clxc, cuxc)
ref_image_copy.remove_background(32)
ref_image_copy.downweight_edges()
ref_image_copy.compute_projections()

# instanciate the donuts.image.Image class with the masked modified data (explosions?)
ref_image_masked = Image(data_copy_ma, header)
clyc, cuyc, clxc, cuxc = ref_image_masked.calculate_image_geometry(20, 20, 'x', 65, 32)
ref_image_masked.trim(clyc, cuyc, clxc, cuxc)
ref_image_masked.remove_background(32)
ref_image_masked.downweight_edges()
ref_image_masked.compute_projections()

# plot the projections
# note the masked section if offset from 505, 505 as it's been added to the original data before prescan correction
fig, ax = plt.subplots(3, figsize=(10, 10))
ax[0].plot(ref_image_orig.proj_x, 'r-', ref_image_orig.proj_y, 'b-')
ax[1].plot(ref_image_copy.proj_x, 'r-', ref_image_copy.proj_y, 'b-')
ax[2].plot(ref_image_masked.proj_x, 'r-', ref_image_masked.proj_y, 'b-')
plt.show()

### Now loop over some images and do some calculations with and without masking

In [ ]:
for img in imgs[1:]:
    # load the check image data
    with fits.open(img) as ff:
        cdata = ff[0].data
        cheader = ff[0].header
    
    # make the check image object for unmasked data
    # NOTE: we just use the same image geometry from above
    check_image_orig = Image(cdata, cheader)
    check_image_orig.trim(clyo, cuyo, clxo, cuxo)
    check_image_orig.remove_background(32)
    check_image_orig.downweight_edges()
    check_image_orig.compute_projections()
    
    # make the check image object for masked data
    # NOTE: reuse the same mask as above
    cdata_copy = np.copy(cdata)
    cdata_copy_ma = np.ma.array(cdata_copy, mask=data_copy_mask)
    check_image_masked = Image(cdata_copy_ma, cheader)
    check_image_masked.trim(clyo, cuyo, clxo, cuxo)
    check_image_masked.remove_background(32)
    check_image_masked.downweight_edges()
    check_image_masked.compute_projections()
    
    # compute the offset for original data
    check_image_orig.compute_offset(ref_image_orig)
    print(f"{check_image_orig.x} {check_image_orig.y}")
    
    # compute the offset for the unmasked bad data
    check_image_orig.compute_offset(ref_image_copy)
    print(f"{check_image_orig.x} {check_image_orig.y}")
    
    # compute the offset for masked data
    check_image_masked.compute_offset(ref_image_masked)
    print(f"{check_image_masked.x} {check_image_masked.y}")
    
    print('\n\n')

# Testing masked arrays in Donuts

In [3]:
ref_img = imgs[0]

In [4]:
help(Donuts)

Help on class Donuts in module donuts.donuts:

class Donuts(builtins.object)
 |  Donuts(refimage, image_ext=0, exposure='EXPTIME', normalise=True, subtract_bkg=True, downweight_edges=True, prescan_width=0, overscan_width=0, scan_direction='x', border=64, ntiles=32, calculation_area_override=None, image_pixel_mask=None, image_class=<class 'donuts.image.Image'>)
 |  
 |  This class provides methods for measuring shifts between
 |  a series of images of the same star field. First we initialise
 |  the object and generate a reference image. Subsequent images are
 |  aligned to this frame of this reference image.
 |  
 |  Attributes
 |  ----------
 |  None
 |  
 |  Methods defined here:
 |  
 |  __init__(self, refimage, image_ext=0, exposure='EXPTIME', normalise=True, subtract_bkg=True, downweight_edges=True, prescan_width=0, overscan_width=0, scan_direction='x', border=64, ntiles=32, calculation_area_override=None, image_pixel_mask=None, image_class=<class 'donuts.image.Image'>)
 |      In

In [5]:
import donuts
donuts.__version__

'0.3.5'

In [6]:
# test with no masking
ref = Donuts(ref_img, prescan_width=20, overscan_width=20)

# test with masking
ref_masking = Donuts(ref_img, prescan_width=20, overscan_width=20, image_pixel_mask=np.zeros((2048, 2088)))

/Users/jmcc/.pyenv/versions/miniconda3-latest/lib/python3.7/site-packages/numpy/core/fromnumeric.py:753: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [7]:
for i in imgs[1:]:
    shift = ref.measure_shift(i)
    print(shift.x, shift.y)
    
    shift_mask = ref_masking.measure_shift(i)
    print(shift_mask.x, shift_mask.y)
    print('\n')

-2.2590190632669174 pix -5.811046336956147 pix
-2.2590190632669174 pix -5.811046336956147 pix


-2.6986480518987195 pix -5.927533386487114 pix
-2.6986480518987195 pix -5.927533386487114 pix


-2.896516427580819 pix -5.947785736884461 pix
-2.896516427580819 pix -5.947785736884461 pix


-2.8098509446811155 pix -5.7716209814370405 pix
-2.8098509446811155 pix -5.7716209814370405 pix


-2.8698523615364553 pix -5.6936041804022155 pix
-2.8698523615364553 pix -5.6936041804022155 pix


-2.8207401013627704 pix -5.6395959036772085 pix
-2.8207401013627704 pix -5.6395959036772085 pix


-2.7522377427950353 pix -5.478574993825678 pix
-2.7522377427950353 pix -5.478574993825678 pix


-2.740984309614735 pix -5.564526725941038 pix
-2.740984309614735 pix -5.564526725941038 pix


-2.6771786279498273 pix -5.593855979523206 pix
-2.6771786279498273 pix -5.593855979523206 pix


-2.6146017307069678 pix -5.607589759523061 pix
-2.6146017307069678 pix -5.607589759523061 pix


-2.612328378478984 pix -5.580416628268

In [ ]:
shift.raw_region.mask.shape

In [ ]:
with fits.open(ref_img) as ff:
    data = ff[0].data

In [ ]:
data.shape